In [1]:
import pandas as pd
from IPython.display import clear_output

In [2]:
import time
import random

In [3]:
df = pd.read_parquet('./4YR_cleaned_USAS.parquet')
df.head(2)

,Event,Conf_Rank,Time,Name,School,Standard,Date,Gender,Division,Season,SeasonYear,Time_Adj,Time_Sec,Points,Div_Rank,NCAA_Rank,Power
0,50 Freestyle SCY Male,2,20.20,Tiago ASAKAWA,UMBC,NS,2020-02-14,M,I,19_20,2020,0 days 00:00:20.200000,20.20,664.818090,252.0,291.0,15.96
1,50 Freestyle SCY Male,4,20.72,Niklas Weigelt,UMBC,NS,2020-02-14,M,I,19_20,2020,0 days 00:00:20.720000,20.72,616.009888,542.0,690.0,8.38


In [4]:
df = df.drop('Power',axis=1)
df.head(2)

,Event,Conf_Rank,Time,Name,School,Standard,Date,Gender,Division,Season,SeasonYear,Time_Adj,Time_Sec,Points,Div_Rank,NCAA_Rank
0,50 Freestyle SCY Male,2,20.20,Tiago ASAKAWA,UMBC,NS,2020-02-14,M,I,19_20,2020,0 days 00:00:20.200000,20.20,664.818090,252.0,291.0
1,50 Freestyle SCY Male,4,20.72,Niklas Weigelt,UMBC,NS,2020-02-14,M,I,19_20,2020,0 days 00:00:20.720000,20.72,616.009888,542.0,690.0


In [5]:
name_info = df[['Name', 'School']].drop_duplicates()
name_info.shape
# Add 'Power' column with all values set to -1
name_info['Power'] = -1.0
# Add 'Link' column with all values set to an empty string
name_info['Link'] = ''
name_info['Hometown'] = ''
# Check the result
name_info.head()

,Name,School,Power,Link,Hometown
0,Tiago ASAKAWA,UMBC,-1.0,,
1,Niklas Weigelt,UMBC,-1.0,,
2,Ben Beldner,Binghamton,-1.0,,
3,Justin Meyn,Binghamton,-1.0,,
4,Joshua Franco,NJIT,-1.0,,


In [6]:
import requests
import csv
from bs4 import BeautifulSoup as bs
import pandas as pd
import time as _time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
import re

def getpi(swimmer_name):
    swimmer_name_url = 'https://swimcloud.com/recruiting/rankings/?name='+swimmer_name.replace(' ','+')
    name_url = requests.get(swimmer_name_url, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36', 'Referer' : 'https://google.com/'})
    name_url.encoding = 'utf-8'
    name_soup = bs(name_url.text, 'html.parser')
    # print(name_soup)
    # print(name_soup)
    # print(name_url.text)
    names=[]
    links=[]
    idxs=[]
    teams=[]
    hometowns=[]
    if (name_soup.find('tbody') == None):
        # print('No tbody')
        return None
    swimmer_list = name_soup.find('tbody').find_all('tr')
    i=0
    for swimmer in swimmer_list:
        if len(name_soup.find_all('td', {'class' : 'u-text-end'})) == 0:
            # print('No idx found')
            break
        name = swimmer.find(class_='u-text-semi')
        name_str = str(name)
        start = name_str.find('>') + 1
        end = name_str.find('</a>')
        # Extract the text between '>' and '<'
        name_raw = name_str[start:end]
        if (name_raw.lower() == swimmer_name.lower()):
            names.append(name_raw)
            swimmer_link = name['href'] if name else ''
            links.append(swimmer_link)
            if (swimmer_name == 'Reese Samuel'):
                print(i)
                print(name_soup.find_all('td', {'class' : 'u-text-end'})[i])
            idxs.append(float(name_soup.find_all('td', {'class' : 'u-text-end'})[i].text.strip()))
            team_td = swimmer.find_all('td', class_='u-text-center hidden-xs')[0]
            # Find the <img> tag within the <td>
            img_tag = team_td.find('img')
            if img_tag == None:
                teams.append('')
            else:
                # Extract the 'alt' attribute value, which contains the team name
                team_name = img_tag['alt'][:-5]
                teams.append(team_name)
    
    
            swimmer_name_url = 'https://swimcloud.com'+swimmer_link
            name_url = requests.get(swimmer_name_url, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36', 'Referer' : 'https://google.com/'})
            name_url.encoding = 'utf-8'
            name_soup2 = bs(name_url.text, 'html.parser')
            if (name_soup2.find('ul', class_='o-list-inline o-list-inline--dotted') == None):
                return None
            ul = name_soup2.find('ul', class_='o-list-inline o-list-inline--dotted')
            location_li = ul.find_all('li')[0]
            
            # Extract and print the location text
            location = location_li.text.strip()
            hometowns.append(location)
        i = i + 1
    # print(names)
    # print(links)
    # print(hometowns)
    return [names,links,idxs,teams,hometowns]


In [ ]:
all_names = df.Name.unique()
j = 13000
modified = 0

for j in range(len(all_names)):
    name = all_names[j]
    # print('about to parse ' + name)
    if (j > -1):
        lists = getpi(name)
        if (lists == None):
            print('No tbody')
            time.sleep(60)
            j = j - 1
        else:
            if len(lists[0]) == 1:
                name_info.loc[(name_info.Name == name), 'Power'] = lists[2][0]
                name_info.loc[(name_info.Name == name), 'Link'] = lists[1][0]
                name_info.loc[(name_info.Name == name), 'Hometown'] = lists[4][0]
                print(" Power " + str(lists[2][0]))
                modified += 1
            elif len(lists[0]) > 1:
                print(name)
                x_old = df[df.Name == name]['School'].value_counts()
                x_old = x_old[x_old > 0]
                
                for school in x_old.index:
                    k = 0
                    for school2 in lists[3]:
                        if (school in school2 or school2 in school and school2 != ''):
                            name_info.loc[(name_info.Name == name) & (name_info.School == school), 'Power'] = lists[2][k]
                            name_info.loc[(name_info.Name == name) & (name_info.School == school), 'Link'] = lists[1][k]
                            name_info.loc[(name_info.Name == name) & (name_info.School == school), 'Hometown'] = lists[4][k]
                            modified += 1
                            print('Name ' + str(name) + " School " + str(school) + " Power " + str(lists[2][k]))
                        k += 1
            else:
                df.loc[(df.Name == name), 'Power'] = -1

    j = j + 1
    time.sleep(random.uniform(0, 1))
    if (j % 50 == 0):
        clear_output() 
    if (j % 10 == 0):
        print(j)
        print(modified)

600
435
 Power 94.07
Kieran Smith
Name Kieran Smith School Florida Power 1.02
 Power 1.15
 Power 9.17
 Power 10.38
 Power 1.7
 Power 1.09
Liam Bell
Name Liam Bell School Alabama Power 1.76
610
443
Jace Brown
Name Jace Brown School Texas A&M Power 7.95
 Power 9.44
 Power 6.59
 Power 9.98
 Power 9.58
 Power 2.39
 Power 9.03
 Power 7.13
 Power 22.29
 Power 2.45
620
453
 Power 7.69
 Power 1.77
 Power 3.84
